In [ ]:
import json
import pandas as pd

This script was used to format data into something that Label studio could work with. Essentially this was only used in structuring data for the gold labelled validation set.

In [ ]:
# Convert dictionary to a JSON string and write to file
with open('/work/MarkusLundsfrydJensen#1865/Bachelor_project/danish_blame.json', 'w') as file:
    file.write(json.dumps(danish_sentences_with_blame, indent=4))

#here danish_sentences_with_blame is output of function danish_sentences_with_blame_extraction()


#preprocess data for label studio
# Load your data
with open("/work/MarkusLundsfrydJensen#1865/Bachelor_project/danish_blame.json", "r", encoding="utf-8") as f:
    data = json.load(f)

flattened = []

for paragraph, sentences in data.items():
    for sentence_nr, text in sentences.items():
        flattened.append({
            "paragraph": paragraph,
            "sentence_nr": sentence_nr,
            "text": text
        })

# Save in a format Label Studio can import
with open("/work/MarkusLundsfrydJensen#1865/Bachelor_project/labelstudio_data.json", "w", encoding="utf-8") as f:
    json.dump(flattened, f, ensure_ascii=False, indent=2)


In [ ]:
#final data is csv file as output of PolDebate model

meta_data = final_data[['Unnamed: 0','speaker','party']]


meta_data = meta_data.replace({np.nan: None})
meta_data.head()

In [ ]:
#connect label studio data with meta data


# Load the flattened sentence JSON
with open("/work/MarkusLundsfrydJensen#1865/Bachelor_project/labelstudio_data.json", "r", encoding="utf-8") as f:
    sentences = json.load(f)

# Load metadata
meta = meta_data

# Convert metadata to dict for fast lookup
meta_dict = meta.set_index("Unnamed: 0").to_dict(orient="index")

# Merge
for item in sentences:
    paragraph = int(item["paragraph"])
    if paragraph in meta_dict:

        item.update(meta_dict[paragraph])

# Save merged dataset
with open("/work/MarkusLundsfrydJensen#1865/Bachelor_project/labelstudio_merged.json", "w", encoding="utf-8") as f:
    json.dump(sentences, f, ensure_ascii=False, indent=2)


In [ ]:

#Now append preceding and succeding sentence for each blame
import json
# Load the flattened sentence JSON
with open("/work/MarkusLundsfrydJensen#1865/Bachelor_project/labelstudio_merged.json", "r", encoding="utf-8") as f:
    json_data = json.load(f)
#also make government data available

regerings_data = pd.read_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/danish_govs.csv")

final_data["date"] = pd.to_datetime(final_data["date"], format="%Y-%m-%d")


regerings_data["Start Date"] = pd.to_datetime(regerings_data["Start Date"], format="%Y-%m-%d")
regerings_data["End Date"]   = pd.to_datetime(regerings_data["End Date"], format="%Y-%m-%d")


def find_government(date):
    match = regerings_data[(regerings_data["Start Date"] <= date) & (regerings_data["End Date"] >= date)]
    if not match.empty:

        parties = match['Party Letter']
        parties = ast.literal_eval(parties.iloc[0])
        
        return parties
    else:
        print('empty')
        return None



import ast

for entry in json_data:
    paragraph_nr = int(entry['paragraph'])
    sentence_nr = int(entry['sentence_nr'])

    #initialize information on current paragraph
    temp_data = final_data.loc[paragraph_nr]

    #get preceding sentence
    temp_data_da_text = ast.literal_eval(temp_data['da_segmented_text'])
    
    pr_sentence_index = sentence_nr-1

    if pr_sentence_index < 0:
        #do empty string
        pr_sent = 'Not available (first sentence in paragraph)'

    else:
        pr_sent = f'sentence_nr {pr_sentence_index} \n{temp_data_da_text[pr_sentence_index]}'

    # find succesding sentence

    suc_sent_index = sentence_nr + 1

    try:
        suc_sent = f'sentence_nr {suc_sent_index} \n{temp_data_da_text[suc_sent_index]}'

    except:
        suc_sent = 'Not available (last sentence in paragraph)'

    
    #Find out if speaker is member of party in Government
    party = entry['party']
    date = temp_data['date']
    parties_gov = find_government(date)
    if party in parties_gov:
        in_gov = True
    else:
        in_gov = False
    



    #make into dict

    context_dict = {'preceding_sentence': pr_sent, 
                    'succeeding_sent': suc_sent,
                    'current_speaker_in_government': in_gov,
                    'parties_in_government': parties_gov,
                    'date': str(date)
                    }

    #update
    entry.update(context_dict)

